### Aluna: Cristiane Brasil Ueda 

## Função de Rosenbrock 

A função de Rosenbrock para um vetor de $n$ variáveis é definida como:

$$
f(\mathbf{x}) = \sum_{i=1}^{n-1} \left[100(x_{i+1} - x_i^2)^2 + (1 - x_i)^2\right]
$$

Onde:
- $\mathbf{x} = [x_1, x_2, \ldots, x_n]$ é o vetor de variáveis.
- O somatório vai de $i = 1$ até $n-1$, pois cada termo depende de dois elementos consecutivos do vetor.



In [2]:
import numpy as np
import time
from scipy.optimize import minimize

# Combined Rosenbrock function
def rosenbrock(x):
    fval = sum(100.0 * (x[1:] - x[:-1]**2.0)**2.0 + (1 - x[:-1])**2.0)

    xm = x[1:-1]
    xm_m1 = x[:-2]
    xm_p1 = x[2:]
    grad = np.zeros_like(x)
    grad[1:-1] = 200 * (xm - xm_m1**2) - 400 * (xm_p1 - xm**2) * xm - 2 * (1 - xm)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) - 2 * (1 - x[0])
    grad[-1] = 200 * (x[-1] - x[-2]**2)

    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        if i > 0:
            H[i, i - 1] = -400 * x[i - 1]
        if i < n - 1:
            H[i, i] = 1200 * x[i]**2 - 400 * x[i + 1] + 2
            H[i, i + 1] = -400 * x[i]
        else:
            H[i, i] = 200
    for i in range(n - 1):
        H[i + 1, i] = H[i, i + 1]

    return fval, grad, H

# Initial point
x0 = np.array([1.3, 0.7, 0.8, 1.9, 1.2])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: rosenbrock(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: rosenbrock(x)[1], "hess": lambda x: rosenbrock(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: rosenbrock(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Rosenbrock Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Rosenbrock Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   6.6175e-05 |    243 |    N/A |    N/A |       9.59 |     True
BFGS                 |   4.0131e-13 |     30 |     30 |    N/A |       4.00 |     True
Newton-CG            |   1.3408e-02 |   1006 |   1006 |   1000 |     144.97 |    False
trust-ncg            |   3.3223e-04 |   1001 |    848 |    847 |     121.90 |    False
trust-krylov         |   2.7387e-09 |    569 |    569 |    561 |     417.00 |     True
trust-exact          |   8.2289e-10 |    655 |    645 |    655 |     306.62 |     True


$$
\min_{x_1, x_2} \Bigl(\cos x_1 ,\sin x_2 ;-; \frac{x_1}{x_2^2 + 1}\Bigr) $$

In [3]:
def ex02(x):
    x1, x2 = x

    # Função
    f = np.cos(x1) * np.sin(x2) - x1 / (x2**2 + 1)

    # Gradiente
    df_dx1 = -np.sin(x1) * np.sin(x2) - 1 / (x2**2 + 1)
    df_dx2 = np.cos(x1) * np.cos(x2) + (2 * x1 * x2) / (x2**2 + 1)**2
    g = np.array([df_dx1, df_dx2])

    # Hessiana
    d2f_dx1dx1 = -np.cos(x1) * np.sin(x2)
    d2f_dx1dx2 = -np.sin(x1) * np.cos(x2) + (2 * x2) / (x2**2 + 1)**2
    d2f_dx2dx1 = d2f_dx1dx2
    d2f_dx2dx2 = -np.cos(x1) * np.sin(x2) + \
                 (2 * x1 * (x2**2 + 1)**2 - 8 * x1 * x2**2 * (x2**2 + 1)) / (x2**2 + 1)**4
    H = np.array([
        [d2f_dx1dx1, d2f_dx1dx2],
        [d2f_dx2dx1, d2f_dx2dx2]
    ])

    return f, g, H

# Initial point
x0 = np.array([0.0, 0.0])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ex02(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ex02(x)[1], "hess": lambda x: ex02(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ex02(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 02 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 02 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |  -1.0457e+00 |    167 |    N/A |    N/A |      19.88 |     True
BFGS                 |  -9.6673e+22 |    248 |    236 |    N/A |      39.83 |    False
Newton-CG            |   0.0000e+00 |      1 |      1 |      1 |       4.18 |     True
trust-ncg            |  -1.6960e+05 |    399 |    396 |    395 |      40.51 |    False
trust-krylov         |   0.0000e+00 |      2 |      2 |      1 |       4.08 |    False
trust-exact          |  -2.2003e+05 |    401 |    389 |    401 |     106.46 |    False


Problem Statistics: 

\# of continuous variables: 2

\# of known solutions: 3

Global solution:

Objective function: -2.02181

Continuous variables: $x_1 = 2; x_2 = 0.10578$

In [4]:
results['trust-exact']

{'x0': array([0., 0.]),
 'x*': array([2.20032125e+05, 6.37081271e-04]),
 'fval': np.float64(-220032.03547900406),
 'nfev': 401,
 'njev': 389,
 'nhev': 401,
 'time': 106.45842552185059,
 'success': False}

In [5]:
ex02(np.array([2., 0.10578]))

(np.float64(-2.0218067833370204),
 array([-1.08494061e+00, -1.31240428e-05]),
 array([[ 0.04393797, -0.69731109],
        [-0.69731109,  3.78275021]]))

Testes de derivadas por diferenças finitas

In [11]:
# Gradiente e Hessiana numéricas
from scipy.optimize import approx_fprime
def numerical_hessian(f_grad, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    fx = f_grad(x)
    for i in range(n):
        x1 = x.copy()
        x1[i] += h
        f1 = f_grad(x1)
        x2 = x.copy()
        x2[i] -= h
        f2 = f_grad(x2)
        H[:, i] = (f1 - f2) / (2 * h)
    return H

# Ponto de teste
x0 = np.array([1.0, 1.0])
eps = np.sqrt(np.finfo(float).eps)

# Avaliação da função
f_val, g_analytical, H_analytical = ex02(x0)

# Gradiente numérico
g_numeric = approx_fprime(x0, lambda x: ex02(x)[0], eps)

# Hessiana numérica via gradiente
H_numeric = numerical_hessian(lambda x: ex02(x)[1], x0)

# Diferenças absolutas
grad_diff = np.abs(g_numeric - g_analytical)
hess_diff = np.abs(H_numeric - H_analytical)

grad_diff, hess_diff


(array([7.76445575e-09, 1.14897460e-08]),
 array([[2.34168240e-12, 5.61722197e-12],
        [5.61722197e-12, 5.45773426e-11]]))

$$
\min_{x, y} 
\Bigl[
1 
+ (x + y + 1)^2 \,\bigl(19 - 14x + 3x^2 - 14y + 6xy + 3y^2\bigr)
\Bigr]
\;\times\;
\Bigl[
30 
+ (2x - 3y)^2 \,\bigl(18 - 32x^2 + 12x^2 + 48y - 36xy + 27y^2\bigr)
\Bigr]
$$

In [6]:
import numpy as np

# Função e gradiente analítico
def goldstein_price_fg(x):
    x1, x2 = x
    a = x1 + x2 + 1
    b = 2 * x1 - 3 * x2

    A = 1 + a**2 * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2)
    B = 30 + b**2 * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2)

    f = A * B

    dA_dx1 = 2 * a * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2) \
             + a**2 * (-14 + 6 * x1 + 6 * x2)
    dA_dx2 = 2 * a * (19 - 14 * x1 + 3 * x1**2 - 14 * x2 + 6 * x1 * x2 + 3 * x2**2) \
             + a**2 * (-14 + 6 * x1 + 6 * x2)

    dB_dx1 = 4 * b * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2) \
             + b**2 * (-32 + 24 * x1 - 36 * x2)
    dB_dx2 = -6 * b * (18 - 32 * x1 + 12 * x1**2 + 48 * x2 - 36 * x1 * x2 + 27 * x2**2) \
             + b**2 * (48 - 36 * x1 + 54 * x2)

    df_dx1 = dA_dx1 * B + A * dB_dx1
    df_dx2 = dA_dx2 * B + A * dB_dx2
    g = np.array([df_dx1, df_dx2])

    return f, g

# Hessiana numérica com diferenças centrais
def numerical_hessian(grad_func, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += h
        x2[i] -= h
        g1 = grad_func(x1)
        g2 = grad_func(x2)
        H[:, i] = (g1 - g2) / (2 * h)
    return H

# Interface principal
def ex03(x):
    f, g = goldstein_price_fg(x)
    H = numerical_hessian(lambda x_: goldstein_price_fg(x_)[1], x)
    return f, g, H



In [7]:
x0 = np.array([0.0, -1.0])
fval, grad, hess = ex03(x0)
print("f(x) =", fval)
print("∇f(x) =", grad)
print("∇²f(x) =\n", hess)

f(x) = 3.0
∇f(x) = [0. 0.]
∇²f(x) =
 [[ 504.0000021  -215.99999929]
 [-216.00000055  864.00000477]]


In [8]:
# Initial point
x0 = np.array([0.0, 0.0])

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ex03(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ex03(x)[1], "hess": lambda x: ex03(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ex03(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 03 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 03 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.0000e+01 |    121 |    N/A |    N/A |      11.10 |     True
BFGS                 |   3.0000e+01 |     16 |     16 |    N/A |       3.00 |     True
Newton-CG            |   3.0000e+01 |     13 |     13 |      8 |       3.00 |     True
trust-ncg            |   3.0000e+01 |     11 |      9 |      8 |       2.39 |     True
trust-krylov         |   3.0000e+01 |      7 |      7 |      6 |       6.04 |     True
trust-exact          |   3.0000e+01 |      7 |      7 |      7 |       2.00 |     True


Problem Statistics: 

\# of continuous variables: 2

\# of known solutions: 4

Global solution:

Objective function: 3

Continuous variables: $x_1 = 0.0; x_2 = -1.0$

$$
f_1(x) \;=\; -20 \, e^{-0.2 \,\sqrt{\frac{1}{D}\,\sum_{i=1}^{D} x_i^2}} 
\;-\; e^{\frac{1}{D}\,\sum_{i=1}^{D} \cos\bigl(2\pi x_i\bigr)} 
\;+\; 20 
\;+\; e
$$

In [9]:

# Valor e gradiente da função Ackley 1
def ackley1_fg(x):
    D = len(x)
    sum_sq = np.sum(x**2)
    sum_cos = np.sum(np.cos(2 * np.pi * x))
    
    term1 = -20 * np.exp(-0.2 * np.sqrt(sum_sq / D))
    term2 = -np.exp(sum_cos / D)
    f = term1 + term2 + 20 + np.e

    # Gradiente analítico
    sqrt_sum_sq = np.sqrt(sum_sq / D)
    if sqrt_sum_sq == 0:
        grad1 = 0
    else:
        grad1 = (4 * x / (D * sqrt_sum_sq)) * np.exp(-0.2 * sqrt_sum_sq)

    grad2 = (2 * np.pi / D) * np.sin(2 * np.pi * x) * np.exp(sum_cos / D)

    g = grad1 + grad2
    return f, g

# Hessiana numérica usando gradiente externo
def numerical_hessian(grad_func, x, h=1e-5):
    n = len(x)
    H = np.zeros((n, n))
    for i in range(n):
        x1 = x.copy()
        x2 = x.copy()
        x1[i] += h
        x2[i] -= h
        g1 = grad_func(x1)
        g2 = grad_func(x2)
        H[:, i] = (g1 - g2) / (2 * h)
    return H

# Interface final
def ackley1(x):
    f, g = ackley1_fg(x)
    H = numerical_hessian(lambda x_: ackley1_fg(x_)[1], x)
    return f, g, H



In [10]:
x0 = np.zeros(5)
fval, grad, hess = ackley1(x0)
print("f(x) =", fval)
print("∇f(x) =", grad)
print("∇²f(x) =\n", hess)

f(x) = 4.440892098500626e-16
∇f(x) = [0. 0. 0. 0. 0.]
∇²f(x) =
 [[178906.74089307      0.              0.              0.
       0.        ]
 [     0.         178906.74089307      0.              0.
       0.        ]
 [     0.              0.         178906.74089307      0.
       0.        ]
 [     0.              0.              0.         178906.74089307
       0.        ]
 [     0.              0.              0.              0.
  178906.74089307]]


In [11]:
# Initial point
x0 = np.ones(5)

# List of methods to compare
methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: ackley1(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: ackley1(x)[1], "hess": lambda x: ackley1(x)[2]},
}

# Executa as otimizações e armazena resultados
results = {}

for name, opts in methods.items():
    start_time = time.time()
    res = minimize(lambda x: ackley1(x)[0], x0, **opts, options={"disp": False})
    elapsed_time = (time.time() - start_time) * 1000  # em ms
    results[name] = {
        "x0": x0,
        "x*": res.x,
        "fval": res.fun,
        "nfev": res.nfev,
        "njev": res.get("njev", None),
        "nhev": res.get("nhev", None),
        "time": elapsed_time,
        "success": res.success
    }

# Apresenta os resultados com alinhamento
print("\n==== Example 03 Results ====\n")
header = f"{'Method':<20} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Time (ms)':>10} | {'Success':>8}"
print(header)
print("-" * len(header))

for method, data in results.items():
    print(f"{method:<20} | {data['fval']:12.4e} | {data['nfev']:6d} | "
          f"{data['njev'] if data['njev'] is not None else '  N/A':>6} | "
          f"{data['nhev'] if data['nhev'] is not None else '  N/A':>6} | "
          f"{data['time']:10.2f} | {str(data['success']):>8}")


==== Example 03 Results ====

Method               |        f(x*) |   nfev |   njev |   nhev |  Time (ms) |  Success
--------------------------------------------------------------------------------------
Nelder-Mead          |   3.5745e+00 |    145 |    N/A |    N/A |      35.26 |     True
BFGS                 |   3.5745e+00 |      7 |      7 |    N/A |       5.19 |     True
Newton-CG            |   3.5745e+00 |      4 |      4 |      3 |       2.92 |     True
trust-ncg            |   3.5745e+00 |      3 |      3 |      2 |       2.00 |     True
trust-krylov         |   3.5745e+00 |      3 |      3 |      2 |       3.00 |     True
trust-exact          |   3.5745e+00 |      3 |      3 |      3 |       4.00 |     True


## Exercício 1 - Função 105 - Rosenbrock

Avaliar para diferentes valores de D



$$
f_{105}(\mathbf{x}) = \sum_{i=1}^{D-1} \left[ 100(x_{i+1} - x_i^2)^2 + (x_i - 1)^2 \right]
$$


In [14]:
import numpy as np
import time
from scipy.optimize import minimize

# Função de Rosenbrock com gradiente e hessiana
def f105_rosenbrock(x):
    fval = np.sum(100.0 * (x[1:] - x[:-1]**2)**2 + (x[:-1] - 1)**2)

    grad = np.zeros_like(x)
    grad[0] = -400 * x[0] * (x[1] - x[0]**2) + 2 * (x[0] - 1)
    for i in range(1, len(x)-1):
        grad[i] = 200 * (x[i] - x[i-1]**2) - 400 * x[i] * (x[i+1] - x[i]**2) + 2 * (x[i] - 1)
    grad[-1] = 200 * (x[-1] - x[-2]**2)

    n = len(x)
    H = np.zeros((n, n))
    for i in range(n - 1):
        H[i, i] += 1200 * x[i]**2 - 400 * x[i+1] + 2
        H[i, i+1] += -400 * x[i]
        H[i+1, i] += -400 * x[i]
    H[-1, -1] += 200

    return fval, grad, H

# Configurações
dimensions = [2, 5, 10, 20]
n_runs = 10

methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS", "jac": lambda x: f105_rosenbrock(x)[1]},
    "Newton-CG": {"method": "Newton-CG", "jac": lambda x: f105_rosenbrock(x)[1], "hess": lambda x: f105_rosenbrock(x)[2]},
    "trust-ncg": {"method": "trust-ncg", "jac": lambda x: f105_rosenbrock(x)[1], "hess": lambda x: f105_rosenbrock(x)[2]},
    "trust-krylov": {"method": "trust-krylov", "jac": lambda x: f105_rosenbrock(x)[1], "hess": lambda x: f105_rosenbrock(x)[2]},
    "trust-exact": {"method": "trust-exact", "jac": lambda x: f105_rosenbrock(x)[1], "hess": lambda x: f105_rosenbrock(x)[2]},
}

results_all = []

# Execução
for D in dimensions:
    x0 = np.random.uniform(-2, 2, D)

    for name, opts in methods.items():
        times = []
        last_res = None

        for _ in range(n_runs):
            try:
                start = time.time()
                res = minimize(lambda x: f105_rosenbrock(x)[0], x0, **opts, options={"maxiter": 10000, "disp": False})
                elapsed = (time.time() - start) * 1000
                times.append(elapsed)
                last_res = res
            except Exception as e:
                print(f"[Erro] {name} D={D}: {e}")
                times.append(np.nan)

        if last_res is not None:
            results_all.append({
                "Método": name,
                "D": D,
                "f(x*)": last_res.fun,
                "nfev": last_res.nfev,
                "njev": last_res.get("njev", None),
                "nhev": last_res.get("nhev", None),
                "Tempo (ms)": np.nanmean(times),
                "Sucesso": last_res.success
            })

# Impressão dos resultados
print("\n==== Resultados - Comparação Multi-Dimensional (Função Rosenbrock) ====\n")
header = f"{'Método':<20} | {'D':>3} | {'f(x*)':>12} | {'nfev':>6} | {'njev':>6} | {'nhev':>6} | {'Tempo (ms)':>10} | {'Sucesso':>8}"
print(header)
print("-" * len(header))

for r in results_all:
    print(f"{r['Método']:<20} | {r['D']:>3} | {r['f(x*)']:12.4e} | {r['nfev']:6d} | "
          f"{r['njev'] if r['njev'] is not None else '  N/A':>6} | "
          f"{r['nhev'] if r['nhev'] is not None else '  N/A':>6} | "
          f"{r['Tempo (ms)']:10.2f} | {str(r['Sucesso']):>8}")



==== Resultados - Comparação Multi-Dimensional (Função Rosenbrock) ====

Método               |   D |        f(x*) |   nfev |   njev |   nhev | Tempo (ms) |  Sucesso
--------------------------------------------------------------------------------------------
Nelder-Mead          |   2 |   9.6159e-10 |    152 |    N/A |    N/A |       4.95 |     True
BFGS                 |   2 |   1.0522e-17 |     31 |     31 |    N/A |       3.18 |     True
Newton-CG            |   2 |   4.7684e-14 |     34 |     34 |     28 |       3.79 |     True
trust-ncg            |   2 |   4.3588e-13 |     25 |     23 |     22 |       2.15 |     True
trust-krylov         |   2 |   2.8553e-14 |     22 |     22 |     19 |      19.88 |     True
trust-exact          |   2 |   6.9387e-12 |      6 |      6 |      6 |       1.23 |     True
Nelder-Mead          |   5 |   1.4440e-09 |    968 |    N/A |    N/A |      30.28 |     True
BFGS                 |   5 |   7.5502e-15 |     62 |     62 |    N/A |       6.36 |     T

## Exercício 2- Função 114 - Scahffer 3

Testar para diferentes pontos iniciais.

$$
f_(114)(x_1, x_2) = 0.5 + 
\frac{\cos^2\left(x_1 \cdot \sin(x_2)\right) - 0.5}
{\left[1 + 0.001\left(x_1^2 + x_2^2\right)\right]^2}
$$


In [16]:
import numpy as np
import time
from scipy.optimize import minimize

# Função Scalifier f_114

def scalifier114(x):
    x1, x2 = x[0], x[1]
    numerator = np.cos(x1 * np.sin(x2))**2 - 0.5
    denominator = (1 + 0.001 * (x1**2 + x2**2))**2
    return 0.5 + numerator / denominator

# Configurações
dist_opt = lambda x: np.linalg.norm(x - np.array([0, 1.253115]))
n_runs = 10
bounds = [(-100, 100), (-100, 100)]
initial_points = [
    np.array([-25.09197623,  90.14286128]),
    np.array([46.39878836, 19.73169684]),
    np.array([-68.79627191, -68.80109593]),
    np.array([-88.38327757,  73.23522915]),
    np.array([20.22300235, 41.61451556]),
    np.array([-95.88310114,  93.98197043])
]

methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS"},
    "L-BFGS-B": {"method": "L-BFGS-B", "bounds": bounds},
    "trust-constr": {"method": "trust-constr", "bounds": bounds},
}

# Teste para diferentes pontos iniciais
results = []

for i, x0 in enumerate(initial_points):
    print(f"\n=== Teste com ponto inicial {i+1}: {x0} ===")

    for name, opts in methods.items():
        times, solutions, func_values, successes = [], [], [], []

        for _ in range(n_runs):
            try:
                perturbed_x0 = x0 + np.random.normal(0, 0.1, size=2)
                start_time = time.time()
                res = minimize(scalifier114, perturbed_x0, **opts,
                               options={'maxiter': 10000, 'disp': False})
                elapsed_time = (time.time() - start_time) * 1000

                times.append(elapsed_time)
                solutions.append(res.x)
                func_values.append(res.fun)
                successes.append(res.success)

            except Exception as e:
                print(f"Erro no método {name}: {str(e)}")
                times.append(np.nan)
                solutions.append(np.array([np.nan, np.nan]))
                func_values.append(np.nan)
                successes.append(False)

        valid_runs = [i for i in range(n_runs) if successes[i]]
        if valid_runs:
            avg_time = np.mean([times[i] for i in valid_runs])
            avg_solution = np.mean([solutions[i] for i in valid_runs], axis=0)
            avg_func = np.mean([func_values[i] for i in valid_runs])
            best_func = np.min([func_values[i] for i in valid_runs])
            best_solution = solutions[np.argmin([func_values[i] for i in valid_runs])]

            results.append({
                'Initial Point': f"{x0[0]:.1f}, {x0[1]:.1f}",
                'Method': name,
                'Avg Time (ms)': avg_time,
                'Success Rate': f"{len(valid_runs)/n_runs*100:.1f}%",
                'Avg f(x*)': avg_func,
                'Best f(x*)': best_func,
                'Best Solution': f"[{best_solution[0]:.6f}, {best_solution[1]:.6f}]",
                'Distance to Optimum': dist_opt(best_solution)
            })

# Impressão detalhada
deprint = lambda x: print(x)
deprint("\n=== RESULTADOS DETALHADOS ===")
deprint("="*120)
deprint("{:<15} {:<15} {:<15} {:<15} {:<15} {:<15} {:<25} {:<15}".format(
    'Ponto Inicial', 'Método', 'Tempo Médio', 'Taxa Sucesso', 'Média f(x*)', 
    'Melhor f(x*)', 'Melhor Solução', 'Distância Ótimo'))
deprint("-"*120)

for res in results:
    deprint("{:<15} {:<15} {:<15.2f} {:<15} {:<15.6f} {:<15.6f} {:<25} {:<15.6f}".format(
        res['Initial Point'], res['Method'], res['Avg Time (ms)'], res['Success Rate'],
        res['Avg f(x*)'], res['Best f(x*)'], res['Best Solution'], res['Distance to Optimum']))

# Análise por método
deprint("\n=== ANÁLISE POR MÉTODO ===")
deprint("="*90)
deprint("{:<15} {:<15} {:<15} {:<15} {:<15}".format(
    'Método', 'Média Tempo', 'Taxa Sucesso', 'Melhor f(x*)', 'Distância Média'))
deprint("-"*90)

for method in methods.keys():
    method_results = [r for r in results if r['Method'] == method]
    if method_results:
        avg_time = np.mean([r['Avg Time (ms)'] for r in method_results])
        avg_success = np.mean([float(r['Success Rate'][:-1]) for r in method_results])
        best_f = np.min([r['Best f(x*)'] for r in method_results])
        avg_dist = np.mean([r['Distance to Optimum'] for r in method_results])

        deprint("{:<15} {:<15.2f} {:<15.1f}% {:<15.6f} {:<15.6f}".format(
            method, avg_time, avg_success, best_f, avg_dist))



=== Teste com ponto inicial 1: [-25.09197623  90.14286128] ===


c:\Users\Tikara\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)



=== Teste com ponto inicial 2: [46.39878836 19.73169684] ===

=== Teste com ponto inicial 3: [-68.79627191 -68.80109593] ===

=== Teste com ponto inicial 4: [-88.38327757  73.23522915] ===

=== Teste com ponto inicial 5: [20.22300235 41.61451556] ===

=== Teste com ponto inicial 6: [-95.88310114  93.98197043] ===

=== RESULTADOS DETALHADOS ===
Ponto Inicial   Método          Tempo Médio     Taxa Sucesso    Média f(x*)     Melhor f(x*)    Melhor Solução            Distância Ótimo
------------------------------------------------------------------------------------------------------------------------
-25.1, 90.1     Nelder-Mead     4.82            100.0%          0.493726        0.488320        [-23.742175, 70.561294]   73.261958      
-25.1, 90.1     BFGS            10.64           100.0%          0.494377        0.494206        [-17.860277, 89.279041]   89.819559      
-25.1, 90.1     L-BFGS-B        8.48            100.0%          0.494508        0.494359        [-20.844967, 89.332665

## Exercício 3 - Função 142 - Streched V Senoidal Modificada

Avaliar para diferentes valores de D

$$
f142(\mathbf{x}) = \sum_{i=1}^{D-1} 
\left( 0.5 x_i^2 + 1.5 x_{i+1}^2 \right)^{0.3} \cdot 
\left[ \sin^2 \left( 40 \cdot \left| x_i - x_{i+1} \right|^{0.2} \right) + 0.05 \right]
$$

In [21]:
import numpy as np
import time
from scipy.optimize import minimize

def f142(x):
    D = len(x)
    fval = 0.0
    for i in range(D-1):
        term1 = 0.5 * x[i]**2 + 1.5 * x[i+1]**2
        term2 = np.sin(40 * abs(x[i] - x[i+1])**0.2)**2 + 0.05
        fval += term1**0.3 * term2
    return fval

# Configurações
dimensions = [2, 5, 10]
n_runs = 10
bounds = [(-5, 15)]  # Novo intervalo

methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS"},
    "L-BFGS-B": {"method": "L-BFGS-B", "bounds": bounds},
    "trust-constr": {"method": "trust-constr", "bounds": bounds},
}

all_results = []

# Define uma seed para reprodutibilidade
np.random.seed(42)

for D in dimensions:
    current_bounds = bounds * D
    
    for name, opts in methods.items():
        times = []
        func_vals = []
        distances = []
        successes = 0
        
        if opts["method"] in ["L-BFGS-B", "trust-constr"]:
            opts["bounds"] = current_bounds
        
        for _ in range(n_runs):
            try:
                x0 = np.random.uniform(low=-5, high=15, size=D)  # Novo intervalo
                
                start_time = time.time()
                res = minimize(f142, x0, **opts, options={'maxiter': 5000, 'disp': False})
                elapsed_time = (time.time() - start_time) * 1000
                
                valid = all(-5 <= xi <= 15 for xi in res.x)
                
                times.append(elapsed_time)
                func_vals.append(res.fun if valid else np.nan)
                distances.append(np.linalg.norm(res.x) if valid else np.nan)
                successes += int(res.success and valid)
                
            except Exception as e:
                print(f"Erro no método {name} (D={D}): {str(e)}")
                times.append(np.nan)
                func_vals.append(np.nan)
                distances.append(np.nan)
        
        avg_time = np.nanmean(times)
        std_time = np.nanstd(times)
        avg_func = np.nanmean(func_vals)
        std_func = np.nanstd(func_vals)
        avg_dist = np.nanmean(distances)
        success_rate = successes/n_runs*100
        
        all_results.append({
            'Method': name,
            'D': D,
            'f(x*)': f"{avg_func:.4e} ± {std_func:.2e}",
            'Time (ms)': f"{avg_time:.1f} ± {std_time:.1f}",
            'Success Rate (%)': success_rate,
            'Distance to Optimum': f"{avg_dist:.4e}",
            'Bounds': "[-5,15]"
        })

# Resultados
print("\n==== Resultados para f142(x) com bounds [-5, 15] ====")
print("{:<15} {:<5} {:<20} {:<20} {:<20} {:<20} {:<10}".format(
    'Method', 'D', 'f(x*)', 'Time (ms)', 'Success Rate (%)', 
    'Distance to Optimum', 'Bounds'))
print("-"*120)

for res in all_results:
    print("{:<15} {:<5} {:<20} {:<20} {:<20} {:<20} {:<10}".format(
        res['Method'],
        res['D'],
        res['f(x*)'],
        res['Time (ms)'],
        f"{res['Success Rate (%)']:.1f}",
        res['Distance to Optimum'],
        res['Bounds']))


c:\Users\Tikara\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)
C:\Users\Tikara\AppData\Local\Temp\ipykernel_29040\4209431183.py:66: RuntimeWarning: Mean of empty slice
  avg_func = np.nanmean(func_vals)
c:\Users\Tikara\AppData\Local\Programs\Python\Python312\Lib\site-packages\numpy\lib\_nanfunctions_impl.py:2053: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\Tikara\AppData\Local\Temp\ipykernel_29040\4209431183.py:68: RuntimeWarning: Mean of empty slice
  avg_dist = np.nanmean(distances)
c:\Users\Tikara\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_different


==== Resultados para f142(x) com bounds [-5, 15] ====
Method          D     f(x*)                Time (ms)            Success Rate (%)     Distance to Optimum  Bounds    
------------------------------------------------------------------------------------------------------------------------
Nelder-Mead     2     6.9008e-02 ± 4.97e-02 1.8 ± 0.5            70.0                 2.8436e+00           [-5,15]   
BFGS            2     1.1284e-01 ± 3.89e-02 2.6 ± 0.7            70.0                 5.3489e+00           [-5,15]   
L-BFGS-B        2     1.3149e-01 ± 7.09e-02 3.4 ± 1.6            100.0                6.4295e+00           [-5,15]   
trust-constr    2     1.4922e-01 ± 7.62e-02 44.7 ± 15.4          100.0                7.4769e+00           [-5,15]   
Nelder-Mead     5     6.8571e-01 ± 9.89e-02 10.7 ± 3.9           40.0                 1.4694e+01           [-5,15]   
BFGS            5     5.8045e-01 ± 7.40e-02 7.0 ± 3.4            30.0                 1.5084e+01           [-5,15]   

## Exercício 4 - Função 61 - Hansen

A partir de diferentes pontos iniciais, encontrar ao menos dois dos mínimos globais.


$$
f_{61}(x) = \left( \sum_{i=0}^{4} (i+1)\cos\left((i+1)x_1 + i\right) \right) \cdot \left( \sum_{j=0}^{4} (j+1)\cos\left((j+2)x_2 + j\right) \right)
$$


In [23]:
import numpy as np
import time
from scipy.optimize import minimize

def f61(x):
    x1, x2 = x[0], x[1]
    sum1 = sum((i+1) * np.cos((i+1)*x1 + i) for i in range(0, 5))
    sum2 = sum((j+1) * np.cos((j+2)*x2 + j) for j in range(0, 5))
    return sum1 * sum2

# Novos pontos iniciais diferentes dos do colega
initial_points = [
    np.array([-2.0, 7.0]),
    np.array([6.0, 6.0]),
    np.array([-9.0, 9.0]),
    np.array([3.0, -8.0]),
    np.random.uniform(-10, 10, 2),
    np.random.uniform(-10, 10, 2)
]

methods = {
    "Nelder-Mead": {"method": "Nelder-Mead"},
    "BFGS": {"method": "BFGS"},
    "L-BFGS-B": {"method": "L-BFGS-B", "bounds": [(-10, 10), (-10, 10)]},
    "trust-constr": {"method": "trust-constr", "bounds": [(-10, 10), (-10, 10)]}
}

results = []
found_minima = []

n_runs = 10
tolerance = 0.5

for idx, x0 in enumerate(initial_points):
    print(f"\n=== Ponto Inicial {idx+1}: {x0} ===")
    
    for method_name, opts in methods.items():
        best_solutions = []
        times = []
        best_values = []
        
        for _ in range(n_runs):
            perturbed_x0 = x0 + np.random.normal(0, 0.5, size=2)
            
            start = time.time()
            res = minimize(f61, perturbed_x0, **opts, options={'maxiter': 5000, 'disp': False})
            duration = (time.time() - start) * 1000
            
            if res.success:
                sol = np.round(res.x, decimals=4)
                times.append(duration)
                best_values.append(res.fun)
                best_solutions.append(sol.tolist())
        
        unique_minima = []
        for sol in best_solutions:
            if not any(np.linalg.norm(np.array(sol) - np.array(existing)) < tolerance for existing in unique_minima):
                unique_minima.append(sol)
        
        if unique_minima:
            results.append({
                'Initial Point': x0,
                'Method': method_name,
                'Avg Time (ms)': np.mean(times),
                'Best f(x*)': np.min(best_values),
                'Unique Minima Found': unique_minima
            })

print("\n=== RESULTADOS ===")
print("="*100)
print("{:<15} {:<15} {:<15} {:<20} {}".format("Ponto Inicial", "Método", "Tempo Médio", "Melhor f(x*)", "Mínimos Únicos"))
print("-"*100)

for res in results:
    print("{:<15} {:<15} {:<15.2f} {:<20.6f} {}".format(
        f"[{res['Initial Point'][0]:.1f}, {res['Initial Point'][1]:.1f}]",
        res['Method'],
        res['Avg Time (ms)'],
        res['Best f(x*)'],
        res['Unique Minima Found']
    ))



=== Ponto Inicial 1: [-2.  7.] ===

=== Ponto Inicial 2: [6. 6.] ===

=== Ponto Inicial 3: [-9.  9.] ===

=== Ponto Inicial 4: [ 3. -8.] ===

=== Ponto Inicial 5: [ 6.01173398 -5.99699512] ===

=== Ponto Inicial 6: [-6.65034835 -7.90864319] ===


c:\Users\Tikara\AppData\Local\Programs\Python\Python312\Lib\site-packages\scipy\optimize\_differentiable_functions.py:317: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  self.H.update(self.x - self.x_prev, self.g - self.g_prev)



=== RESULTADOS ===
Ponto Inicial   Método          Tempo Médio     Melhor f(x*)         Mínimos Únicos
----------------------------------------------------------------------------------------------------
[-2.0, 7.0]     Nelder-Mead     1.65            -51.923207           [[-1.5172, 7.7586], [-1.5172, 6.7723], [-0.7562, 8.2449], [-0.7562, 7.2637], [-2.2012, 7.2638]]
[-2.0, 7.0]     BFGS            2.00            -51.923208           [[-0.7562, 7.2638], [-2.7888, 8.7387], [-2.7888, 7.7586], [-2.2012, 7.2638], [-2.2012, 6.2659], [-3.3758, 9.2242], [-2.2012, 8.2449]]
[-2.0, 7.0]     L-BFGS-B        1.48            -63.955689           [[-10.0, 6.7723], [-2.2012, 5.0553], [-2.7888, 5.6806], [-10.0, -10.0], [-1.5172, 7.7586], [-2.2012, 7.2638], [7.4454, -10.0], [-1.5172, 6.7723]]
[-2.0, 7.0]     trust-constr    59.34           -155.369336          [[-1.5172, 5.6806], [-2.2012, 6.2659], [-2.7888, 7.7586], [-2.2012, 7.2638], [-1.5172, 6.7723], [-2.7888, 6.7723], [-0.7562, 7.2638]]
[6.0, 6.0